In [2]:

import warnings
warnings.filterwarnings("ignore")

import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable

import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se

import pathlib
import setup_analysis as sa
import sisepuede as ssp
import support_classes as sc
import support_functions as sf
import importlib
import time
import matplotlib.pyplot as plt
import sql_utilities as sq
from typing import *
import sqlalchemy
import sql_utilities as sqlutil
import time
importlib.reload(ma)
#importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(ml)




<module 'model_electricity' from '/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_electricity.py'>

In [3]:
import model_attributes as ma
import model_variable as mv
import units_manager as um

In [4]:
fp = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/out/sisepuede_summary_results_run_sisepuede_run_2024-02-10T17;45;35.578004/sisepuede_results_sisepuede_run_2024-02-10T17;45;35.578004_WIDE_INPUTS_OUTPUTS.csv"
df_test = pd.read_csv(fp)


# path = pathlib.Path(sa.dir_ref)
# path_fake_data = path.joinpath("fake_data", "fake_data_complete.csv")
# df_fake_data = pd.read_csv(path_fake_data)

def add_missing_lvst_elast(
    df: pd.DataFrame,
) -> pd.DataFrame:
    """
    add buffalo, goat, horse, and mule elasticities to a df
    """
    
    # sheep inputs
    vec_sheep = df["elasticity_lvst_sheep_demand_to_gdppc"].to_numpy()
    
    # buffalo
    vec_buffalo = df["elasticity_lvst_cattle_dairy_demand_to_gdppc"].to_numpy()
    vec_buffalo += df["elasticity_lvst_cattle_nondairy_demand_to_gdppc"].to_numpy()
    vec_buffalo /= 2


    flds_sheep = [
        "elasticity_lvst_goats_demand_to_gdppc",
        "elasticity_lvst_horses_demand_to_gdppc",
        "elasticity_lvst_mules_demand_to_gdppc"
    ]
    for fld in flds_sheep:
        if fld not in df.columns:
            df[fld] = vec_sheep
    
    fld_buffalo = "elasticity_lvst_buffalo_demand_to_gdppc"
    if fld_buffalo not in df.columns:
        df[fld_buffalo] = vec_buffalo

    
    return df

df_test = add_missing_lvst_elast(df_test);


df_base = (
    df_test[
        df_test["primary_id"].isin([14014])
    ]
    .reset_index(drop = True)
)

df_base_input = df_base[
    [x for x in df_base.columns if x not in sa.model_attributes.all_variable_fields_output]
]


    
    

In [5]:
import importlib
importlib.reload(sf)
importlib.reload(mv)
importlib.reload(um)
importlib.reload(ma)


fp = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/docs_new/source/csvs"
path = pathlib.Path(fp)
matt_new = ma.ModelAttributes(
    path, 
    fp_config = path.parents[2].joinpath("sisepuede.config")
)


# modvar = matt_new.dict_variables.get("Volumetric Energy Density")
# modvar.get_property("subsector")
# unit_energy = matt_new.get_unit("energy")
# unit_power = matt_new.get_unit("power")
# unit_power.get_unit_key("none")



modvar = ":math:\\text{CO}_2 Land Use Conversion Emission Factor"
#modvar = "Unadjusted Land Use Transition Probability"
#modvar = "GDP"
#modvar = "Population"
#modvar = "Land Use Yield Reallocation Factor"
modvar = ""
mov = matt_new.dict_variables.get(modvar)


#mov.build_fields(
#    # category_restrictions = ["croplands", "forests_primary"]#"cat_general": ["rural"]}
#    category_restrictions = "croplands"#{"cat_landuse_dim2": ["croplands", "forests_primary"]}
#)



MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


In [184]:
importlib.reload(mv)
importlib.reload(mafl)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(me)
importlib.reload(ml)
importlib.reload(se)

logger = None
matt_new = ma.ModelAttributes(
    path, 
    fp_config = path.parents[2].joinpath("sisepuede.config")
)



importlib.reload(ml)
model_elec = ml.ElectricEnergy(
    matt_new, 
    sa.dir_jl, 
    sa.dir_ref_nemo, 
    logger = None, 
    initialize_julia = True,
);




MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


  Activating project at `~/Documents/Projects/git_jbus/sisepuede/julia`


In [77]:
attrs_check = [x for x in dir(matt_new) if not x.startswith("_")]

attrs_diff = []
for at in attrs_check:
    
    at_sa = getattr(sa.model_attributes, at)
    at_new = getattr(matt_new, at)
    
    if at_sa != at_new:
        attrs_diff.append(at)

In [292]:
importlib.reload(mv)
importlib.reload(mafl)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(me)
importlib.reload(ml)
importlib.reload(se)

logger = None
matt_new = ma.ModelAttributes(
    path, 
    fp_config = path.parents[2].joinpath("sisepuede.config")
)




model_afolu = mafl.AFOLU(matt_new)
model_circecon = mc.CircularEconomy(matt_new);
model_ippu = mi.IPPU(matt_new);
model_elec = ml.ElectricEnergy(
    matt_new, 
    sa.dir_jl, 
    sa.dir_ref_nemo, 
    logger = None, 
    initialize_julia = True,
);

model_energy = me.NonElectricEnergy(matt_new);
model_socioeconomic = se.Socioeconomic(matt_new);


def run_strat(
    df_in: pd.DataFrame,
    project_nemomod: bool = False,
) -> pd.DataFrame:
    
    df_elec = None
    df_cs_integrated = df_in.copy()
    
    df_project_afolu = model_afolu.project(df_cs_integrated)
    df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], matt_new, "concatenate")
    
    # pass integrated df
    df_project_ce = model_circecon.project(df_cs_integrated)
    df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_ce], matt_new, "concatenate")
    
    # pass integrated df
    df_project_ip = model_ippu.project(df_cs_integrated)
    df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_ip], matt_new, "concatenate")
    
    # pass integrated df
    df_project_en = model_energy.project(df_cs_integrated);
    df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_en], matt_new, "concatenate")
    
    df_elec = None
    if project_nemomod:
        print("starting NemoMod")
        os.remove(sa.fp_sqlite_nemomod_db_tmp) if os.path.exists(sa.fp_sqlite_nemomod_db_tmp) else None
        engine = sqlalchemy.create_engine(f"sqlite:///{sa.fp_sqlite_nemomod_db_tmp}")
        t0 = time.time()
        df_elec = model_elec.project(df_cs_integrated, engine = engine, regions = "india")
        t1 = time.time()

    return df_project_afolu, df_cs_integrated, df_elec



dfap, dfc, dfe = run_strat(df_base_input, project_nemomod = True)





MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


  Activating project at `~/Documents/Projects/git_jbus/sisepuede/julia`


In [290]:
#model_elec.model_energy.project_enfu_production_and_demands(dfc);
df_elec_trajectories = dfc
df_reference_capacity_factor = model_elec.dict_nemomod_reference_tables.get(model_elec.model_attributes.table_nemomod_capacity_factor),
df_reference_specified_demand_profile = model_elec.dict_nemomod_reference_tables.get(model_elec.model_attributes.table_nemomod_specified_demand_profile),

regions = None
regions = model_elec.model_attributes.get_region_list_filtered(regions, attribute_region = None)
tuple_enfu_production_and_demands = model_elec.model_energy.project_enfu_production_and_demands(
    df_elec_trajectories, 
    target_energy_units = model_elec.model_attributes.configuration.get("energy_units_nemomod")
)
 
    
dict_out = {}
dict_out.update(
    model_elec.format_nemomod_attribute_table_emission(
        attribute_emission = None
    )
)
# FUEL
dict_out.update(
    model_elec.format_nemomod_attribute_table_fuel(
        attribute_fuel = None
    )
)
# MODEOFOPERATION
dict_out.update(
    model_elec.format_nemomod_attribute_table_mode_of_operation(
        attribute_mode = None
    )
)
# REGION
dict_out.update(
    model_elec.format_nemomod_attribute_table_region(
        attribute_region = None,
        regions = None
    )
)
# STORAGE
dict_out.update(
    model_elec.format_nemomod_attribute_table_storage(
        attribute_storage = None
    )
)
# TECHNOLOGY
dict_out.update(
    model_elec.format_nemomod_attribute_table_technology(
        attribute_technology = None
    )
)
# YEAR
dict_out.update(
    model_elec.format_nemomod_attribute_table_year()
)
# TIMESLICE
dict_out.update(
    model_elec.format_nemomod_table_tsgroup_tables(
        attribute_time_slice = None
    )
)

dict_out.update(
    model_elec.format_nemomod_table_default_parameters(
        attribute_nemomod_table = None
    )
)
# OperationalLife and OperationalLifeStorage
dict_out.update(
    model_elec.format_nemomod_table_operational_life(
        attribute_fuel = None,
        attribute_storage = None,
        attribute_technology = None,
        regions = None
    )
)

dict_out.update(
    model_elec.format_nemomod_table_reserve_margin_tag_fuel(
        regions = None
    )
)
# TechnologyFromStorage and TechnologyToStorage
dict_out.update(
    model_elec.format_nemomod_table_technology_from_and_to_storage(
        attribute_storage = None,
        attribute_technology = None,
        regions = None
    )
)

dict_out.update(
    model_elec.format_nemomod_table_annual_emission_limit(
        df_elec_trajectories,
        attribute_emission = None,
        attribute_time_period = None,
        regions = None
    )
)
# CapitalCostStorage
dict_out.update(
    model_elec.format_nemomod_table_costs_storage(
        df_elec_trajectories,
        regions = None
    )
)
# CapitalCost, FixedCost, and VariableCost -- Costs (Technology) PROBLEM OCCURS HERE
dict_out.update(
    model_elec.format_nemomod_table_costs_technology(
        df_elec_trajectories,
        attribute_fuel = None,
        regions = None
    )
)

In [287]:
model_elec.get_enfu_cats_with_high_dummy_tech_costs(return_type = "dummy_fuel_techs")

['supply_fuel_coal',
 'supply_fuel_crude',
 'supply_fuel_diesel',
 'supply_fuel_electricity',
 'supply_fuel_gasoline',
 'supply_fuel_hydrocarbon_gas_liquids',
 'supply_fuel_hydrogen',
 'supply_fuel_kerosene',
 'supply_fuel_natural_gas',
 'supply_fuel_natural_gas_unprocessed',
 'supply_fuel_oil']

In [198]:
model_elec.model_energy.model_attributes.get_variable_categories("Fraction of Fuel Demand Imported")




['fuel_coal',
 'fuel_crude',
 'fuel_diesel',
 'fuel_electricity',
 'fuel_gasoline',
 'fuel_hydrocarbon_gas_liquids',
 'fuel_hydrogen',
 'fuel_kerosene',
 'fuel_natural_gas',
 'fuel_oil',
 'fuel_coal',
 'fuel_crude',
 'fuel_diesel',
 'fuel_gasoline',
 'fuel_hydrocarbon_gas_liquids',
 'fuel_hydrogen',
 'fuel_kerosene',
 'fuel_natural_gas_unprocessed',
 'fuel_natural_gas',
 'fuel_oil']

In [197]:
model_elec.format_nemomod_table_min_share_production(dfc)

Fraction of Fuel Demand Imported
(56, 3349)


ValueError: shape mismatch: value array of shape (56,10) could not be broadcast to indexing result of shape (56,20)

In [95]:
model_elec.model_attributes.extract_model_variable(#
    me.df_if,
    "Fraction of Fuel Demand Imported",
    expand_to_all_cats = True,
    return_type = "array_base",
    var_bounds = (0, 1)
);

ValueError: shape mismatch: value array of shape (56,10) could not be broadcast to indexing result of shape (56,20)

In [96]:
df_in = me.df_if
modvar = "Fraction of Fuel Demand Imported"
all_cats_missing_val = 0.0
expand_to_all_cats = True
extraction_logic = "all"
force_boundary_restriction: bool = True
include_time_period: bool = False
override_vector_for_single_mv_q: bool = False
return_num_type: type = np.float64
return_type: str = "array_base"
throw_error_on_missing_fields: bool = True
var_bounds = (0, 1)

In [99]:
if (modvar is None) or not isinstance(df_in, pd.DataFrame):
    None

# check model variable specificaiton and data frame
modvar = model_elec.model_attributes.get_variable(modvar)
if modvar is None:
    raise ValueError(f"Invalid variable specified in extract_model_variable: variable '{modvar}' not found.")            

    
# initialize extration fields, which are modified below in different circumstances
flds = modvar.fields

# will default to model variable default value
fill_value = (
    all_cats_missing_val
    if sf.isnumber(all_cats_missing_val)
    else None
)


out = modvar.get_from_dataframe(
    df_in,
    expand_to_all_categories = False,#expand_to_all_cats,
    extraction_logic = extraction_logic,
    fill_value = fill_value,
)

out = out.to_numpy().astype(return_num_type)
out = (
    out[:, 0] 
    if ((len(modvar.fields) == 1) and not override_vector_for_single_mv_q)
    else out
)

In [117]:
# get numeric values and check
var_bounds = [x for x in var_bounds if type(x) in [int, float]]
if len(var_bounds) <= 1:
    raise ValueError(f"Invalid specification of variable bounds '{var_bounds}': there must be a maximum and a minimum numeric value specified.")

# ensure array
out = np.array(out)
b_0, b_1 = np.min(var_bounds), np.max(var_bounds)
m_0, m_1 = np.min(out), np.max(out)


# check bounds
if m_1 > b_1:
    str_warn = f"Invalid maximum value of '{modvar}': specifed value of {m_1} exceeds bound {b_1}."
    if not force_boundary_restriction:
        raise ValueError(str_warn)

    warnings.warn(str_warn + "\nForcing maximum value in trajectory.")

# check min
if m_0 < b_0:
    str_warn = f"Invalid minimum value of '{modvar}': specifed value of {m_0} below bound {b_0}."
    if not force_boundary_restriction:
        raise ValueError(str_warn)

    warnings.warn(str_warn + "\nForcing minimum value in trajectory.")

# force boundary if required
out = sf.vec_bounds(out, var_bounds) if force_boundary_restriction else out
out = np.array([out]).transpose() if (len(out.shape) == 1) else out




In [123]:
model_elec.model_attributes.merge_array_var_partial_cat_to_array_all_cats(
    np.array(out), 
    modvar, 
    missing_vals = all_cats_missing_val,
)

ValueError: shape mismatch: value array of shape (56,10) could not be broadcast to indexing result of shape (56,20)

In [135]:
array_vals = np.array(out)
missing_vals = all_cats_missing_val

output_cats: Union[list, None] = None
output_subsec: Union[str, None] = None

def_missing = 0.0
modvar = model_elec.model_attributes.get_variable(modvar)

subsector = model_elec.model_attributes.get_variable_subsector(modvar)
attr_subsec = model_elec.model_attributes.get_attribute_table(subsector)
cats_restricted = modvar.categories_are_restricted

array_default = np.ones(
            (len(array_vals), attr_subsec.n_key_values)
        )*missing_vals

In [156]:
matt_new2.get_variable_categories("Fraction of Fuel Demand Imported")

['fuel_coal',
 'fuel_crude',
 'fuel_diesel',
 'fuel_electricity',
 'fuel_gasoline',
 'fuel_hydrocarbon_gas_liquids',
 'fuel_hydrogen',
 'fuel_kerosene',
 'fuel_natural_gas',
 'fuel_oil']

In [161]:
modvar.dict_category_keys

{'cat_fuel': ['fuel_coal',
  'fuel_crude',
  'fuel_diesel',
  'fuel_electricity',
  'fuel_gasoline',
  'fuel_hydrocarbon_gas_liquids',
  'fuel_hydrogen',
  'fuel_kerosene',
  'fuel_natural_gas',
  'fuel_oil',
  'fuel_coal',
  'fuel_crude',
  'fuel_diesel',
  'fuel_gasoline',
  'fuel_hydrocarbon_gas_liquids',
  'fuel_hydrogen',
  'fuel_kerosene',
  'fuel_natural_gas_unprocessed',
  'fuel_natural_gas',
  'fuel_oil']}

In [ ]:
modvar = matt_new.dict_variables.get("Unadjusted Land Use Transition Probability")
modvar = matt_new.dict_variables.get(":math:\\text{CH}_4 Stationary Combustion Emission Factor")

#modvar.attribute("unit_mass")
modvar.attribute("unit_area")
modvar.name_fs_safe.upper()


In [5]:
importlib.reload(sf)
importlib.reload(mv)
import time


t0 = time.time()
#modvar = mv.ModelVariable(
#    attr_vars.table.iloc[2],
#    sa.model_attributes.dict_attributes,
#)
t1 = time.time()

"""
vs = mv.VariableSchema(
    modvar.dict_varinfo.get("variable_schema")
)
vs.mutable_elements_clean_ordered
vs.replace({"cat_industry": "cement"}, keys_are_clean = True,)
""";

#modvar.dict_varinfo
#modvar.dict_varinfo





series = attr_vars.table.iloc[2].copy()
series["variable_schema"] = "``ef_ippu_$UNIT-MASS$_$EMISSION-GAS$_per_$UNIT-MASS$_product_use_$CAT-INDUSTRY-DIM1$_$CAT-INDUSTRY-DIM2$_$CAT-TRANSPORTATION$`` (``$UNIT-MASS$ = tonne``, ``$EMISSION-GAS$ = co2``"
series["categories"] = "(``$CAT-INDUSTRY_DIM2$ = paper|cement|plastic``, ``$CAT-INDUSTRY-DIM1$ = product_use_lubricants|product_use_paraffin_wax|cement|plastic``, ``$CAT-TRANSPORTATION$ = aviation|rail_freight|dymm``)"
series["default_value"] = 0.0
#series["categories"] = "(``$CAT-INDUSTRY$ = all``, ``$CAT-TRANSPORTATION$ = aviation|rail_freight|dymm``)"

#series["variable_schema"] = "pij_$CAT-LANDUSE-DIM1$_to_$CAT-LANDUSE-DIM2$"
#series["categories"] = "all"

modvar = mv.ModelVariable(
    series,
    sa.model_attributes.dict_attributes,
)





# get_root_element("CAT-INDUSTRY-DIM4")
#modvar.schema.get_root_element("cat_industry_dim3")
# modvar.schema.mutable_elements_clean_ordered
modvar.dict_category_keys
modvar.schema.mutable_elements_clean_ordered





['cat_industry_dim1', 'cat_industry_dim2', 'cat_transportation']

In [1132]:
regex_at = re.compile("attribute_unit_(.*).csv")
regex_at.match("attribute_unit_temp_gr.csv").groups()



('temp_gr',)

In [1138]:
#sa.model_attributes.all_model_variables
fp = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/docs/source/csvs/attribute_length.csv"
at = AttributeTable(fp, "length", clean_table_fields = True,)
at.table()

,length,unit_length,name,length_equivalent_m,length_equivalent_km,length_equivalent_mile,length_equivalent_mgm,length_equivalent_gm
0,m,``m``,Meters,1.000000e+00,0.00100,6.213730e-04,0.000001,1.000000e-09
1,km,``km``,Kilometers,1.000000e+03,1.00000,6.213700e-01,0.001000,1.000000e-06
2,Mile,``mile``,Miles,1.609340e+03,1.60934,1.000000e+00,0.001609,1.609340e-06
3,Mm,``mgm``,Megameters,1.000000e+06,1000.00000,6.213700e-04,1.000000,1.000000e-03
4,Gm,``gm``,Gigameters,1.000000e+09,1000000.00000,6.213700e-07,1000.000000,1.000000e+00


In [1052]:
modvar(
    df, 
    expand_to_all_categories = False,
    return_type = "array",
)

array([[0.92536768, 0.24845778, 0.37953433, 0.14847882, 0.25139435,
        0.71123234, 0.15856626, 0.48581205, 0.68147416, 0.54829414,
        0.08388734, 0.42894816, 0.66173826, 0.07984463, 0.47521661,
        0.38798217, 0.5266003 , 0.4420342 , 0.88720961, 0.88586851,
        0.70495647, 0.64484206, 0.15106266, 0.60722337],
       [0.92536768, 0.24845778, 0.37953433, 0.14847882, 0.25139435,
        0.71123234, 0.15856626, 0.48581205, 0.68147416, 0.54829414,
        0.08388734, 0.42894816, 0.66173826, 0.07984463, 0.47521661,
        0.38798217, 0.5266003 , 0.4420342 , 0.88720961, 0.88586851,
        0.70495647, 0.64484206, 0.15106266, 0.60722337],
       [0.92536768, 0.24845778, 0.37953433, 0.14847882, 0.25139435,
        0.71123234, 0.15856626, 0.48581205, 0.68147416, 0.54829414,
        0.08388734, 0.42894816, 0.66173826, 0.07984463, 0.47521661,
        0.38798217, 0.5266003 , 0.4420342 , 0.88720961, 0.88586851,
        0.70495647, 0.64484206, 0.15106266, 0.60722337],
       [0.925

In [1026]:
#modvar.name.replace(":math:", "").replace("{", "").replace("\\text", "").replace("}")
df = np.full((20, len(modvar.fields_space)), -1)
df = pd.DataFrame(df, columns = modvar.fields_space)
for fld in modvar.fields:
    df[fld] = np.random.rand()
    
    


In [ ]:
importlib.reload(mv)

series = attr_vars.table.iloc[2].copy()
series["variable_schema"] = "``ef_ippu_$UNIT-MASS$_$EMISSION-GAS$_per_$UNIT-MASS$_product_use_$CAT-INDUSTRY-DIM1$_$CAT-INDUSTRY-DIM2$_$CAT-TRANSPORTATION$`` (``$UNIT-MASS$ = tonne``, ``$EMISSION-GAS$ = co2``"
series["categories"] = "(``$CAT-INDUSTRY_DIM2$ = paper|cement|plastic``, ``$CAT-INDUSTRY-DIM1$ = product_use_lubricants|product_use_paraffin_wax|cement|plastic``, ``$CAT-TRANSPORTATION$ = aviation|rail_freight|dymm``)"

modvar = mv.ModelVariable(
    series,
    sa.model_attributes.dict_attributes,
)




modvar.get_categories_by_element(
    #{
    #    "cat_industry_dim1": ["metals", "plastic", "chemicals"],#"``metals|plastic|chemicals``"
    #},
    dict_category_space = modvar.dict_category_key_space,
)




In [816]:
modvar.schema.get_root_element("cat_indusdtry_dim2")
20000/122.2

163.66612111292963

In [785]:
regex.match("CAT-INDUSTRY-DIM3")
mv.clean_element("-")





'_'

In [666]:
modvar.schema.mutable_elements_ordered
modvar.schema.space_char
re.compile(f".*{modvar.schema.space_char}{modvar.schema.flag_dim}(\d*)$").match("ahitheX-DIM34")





<re.Match object; span=(0, 13), match='ahitheX-DIM34'>

In [527]:
#modvar.schema.get_attribute("emission_gas")
hasattr(attr_cat, "key_values")
#modvar.get_categories_from_string(modvar.dict_varinfo.get(modvar.key_categories), "|", "`")

modvar.get_categories_by_element(
    category_subspace = {
        "cat_industry": "``product_use_lubricants|product_use_paraffin_wax``",
        "cat_transportation": "``product_use_lubricants``|``product_use_paraffin_wax``",
        "cat_transportation": "``product_use_lubricants``|``product_use_paraffin_wax``",
    },
    dict_category_space = modvar.dict_category_keys,
    
)

{'cat_industry': ['product_use_lubricants', 'product_use_paraffin_wax']}

In [390]:
modvar.schema.get_mutable_element(
    "ef_ippu_tonne_co2_per_tonne_production_thisun_jfijdf_clinker",
    "cat_industry"
) 

'thisun_jfijdf'

In [290]:
modvar.build_field_from_elements(
    {"cat_industry": "cement"}, 
    keys_are_clean = True
)



'ef_ippu_tonne_co2_per_tonne_production_cement_clinker'

In [302]:
class Field:
    
    def __init__(self,
        field: str,
        cat: Union[Dict, str, None],
    ):
        
        self.field = field
        self.cat = cat
        
        
    
    def __call__(self,
    ):
        
        return self.field
    

field = Field("ef_ippu_tonne_co2_per_tonne_production_cement_clinker", {"cat_industry": "cement"})

field(), field.cat
        

('ef_ippu_tonne_co2_per_tonne_production_cement_clinker',
 {'cat_industry': 'cement'})

In [278]:
sr = "``ef_ippu_$UNIT-MASS$_$EMISSION-GAS$_per_$UNIT-MASS$_production_$CAT-INDUSTRY-I$_$CAT-TRANSPORTATION-J$`` (``$UNIT-MASS$ = tonne``, ``$EMISSION-GAS$ = ch4``)"
vs2 = mv.VariableSchema(sr)
vs2.mutable_elements_ordered


vs2.replace({"cat_industry_i": "cementitude", "cat_transportation_j": "airplanification"}, keys_are_clean = True,)

'ef_ippu_tonne_ch4_per_tonne_production_cementitude_airplanification'

In [ ]:
vs.mutable_elements_clean_ordered



In [ ]:
sa.model_attributes.dict_attributes.get("abbreviation_subsector").table;

In [244]:
dict1 = {}

dict1[None] = 2
dict1

{None: 2}

In [195]:
def get_variable_schema_base(#self,
    var_schema_raw: str,
) -> str:
    """
    Using a raw variable schema, replace any unit and emission specifications
    """
    
    dict_replacements, schema  = mv.clean_schema(
        var_schema_raw, 
        return_type = "all"
    )
    
    return dict_replacements, schema

dict_replacements, schema = get_variable_schema_base(
    modvar.dict_varinfo.get("variable_schema")
)

In [198]:
isinstance(pd.DataFrame, type)

True

In [201]:
#regex = re.compile("(.*)\$(.*)\$(.*$)")
regex = re.compile("(?<=\$)(.*?)(?=\$)")
regex.findall("this $THIS$ $THIS2$")
#regex = re.search("(?<=\$)(.*?)(?=\$)", "this $HERE$ $NOPE$")





['THIS', ' ', 'THIS2']

In [182]:
string = "this $THIS$ $THIS2$"
string.count("$")

0

In [193]:
modvar.dict_varinfo.get("variable_schema")

'``ef_ippu_$UNIT-MASS$_$EMISSION-GAS$_per_$UNIT-MASS$_production_$CAT-INDUSTRY$`` (``$UNIT-MASS$ = tonne``, ``$EMISSION-GAS$ = ch4``)'

In [215]:
arr_1 = np.array([7, 11, 13, 17, 19, 23, 29])

prs = [2, 3, 5, 7, 11, 13, 17, 19]
arr_2 = []
gaps = []
for i in range(np.prod(prs)):
    check = [i%x for x in prs]
    binary = int(0 in check)
    
    arr_2.append(binary)
    if not binary:
        gaps.append(i)


P = np.prod(prs)
gaps_g1 = [x for x in gaps if x > 1]

classes = []
if False:
    l = []
    for i in range(1, 2):#len(arr_1) + 1):
        subs = list(itertools.combinations(gaps_g1, i))

        l.extend(subs)


    classes = []
    for i, sub in enumerate(l):
        val = np.prod(sub)
        mod = val%P

        classes.append(mod)

    classes = set(classes)
diffs = np.array(gaps[1:]) - np.array(gaps[0:-1])
len([x for x in diffs if x == 4])
w = np.abs(np.array(diffs[1:]) - np.array(diffs[0:-1]))
w = np.where(w == 0)
w;




pi1 = prs[-1]
counts_2 = []

P_prev = np.prod(prs[0:-1])
# check counts of 2 among each subdivision--should be relatively smooth
for i in range(pi1):
    gps = [x for x in gaps if (x >= i*P_prev) & (x < (i + 1)*P_prev)]
    
    ds = np.array(gps[1:]) - np.array(gps[0:-1])
    n = len([x for x in ds if x == 2])
    
    print(n)
    counts_2.append(n)
    #len([x for x in diffs[0:n] if x == 2])
    
    
import sympy
# non-prime nums
gaps_nopr = [x for x in gaps if (not sympy.isprime(x)) and (x > 1)]


l = []
for i in range(1, len(gaps_nopr) + 1):
    subs = list(itertools.combinations(gaps_nopr, i))
    
    l.extend(subs)

classes = []
for i, sub in enumerate(l):
    val = np.prod(sub)
    mod = val%P
    
    classes.append(mod)
    
    
gaps_nopr = [x for x in gaps if (not sympy.isprime(x)) and (x > 1)]
classes = []
for i in range(1, 3):#len(gaps_nopr) + 1):
    combs = itertools.combinations(gaps_nopr, i)
    
    for comb in combs:
        class_check = np.prod(comb)%P
        classes.append(class_check) if (class_check not in classes) else None
    
classes2 = classes.copy()



